<a href="https://colab.research.google.com/github/harishnk28/TransferLearningFineTuning/blob/main/AlexNetModelFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement the ALEXNET model using transfer learning concepts such as ConvNet as fixed feature extractor, Fine-tuning the ConvNet., Pretrained. Select one model from above 3 using following conditions : 

Fine-Tuning for the Large dataset and similar to original dataset 

In [ ]:
#Keras library for CIFAR dataset
from keras.datasets import cifar10

(x_train, y_train),(x_test, y_test)=cifar10.load_data()

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the CIFAR10 dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

170500096/170498071 [==============================] - 2s 0us/step
((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))
((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [ ]:
#Defining the parameters
batch_size= 100
epochs=10
learn_rate=.001

In [ ]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
activation (Activation)      (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 256)         0

In [ ]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
#Defining the parameters
batch_size= 100
epochs=10
learn_rate=.001

In [ ]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [ ]:
#Training the model
AlexNet.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
350/350 [==============================] - 586s 2s/step - loss: 1.6017 - accuracy: 0.4265 - val_loss: 1.9346 - val_accuracy: 0.3323
Epoch 2/10
350/350 [==============================] - 571s 2s/step - loss: 1.3958 - accuracy: 0.5089
Epoch 3/10
350/350 [==============================] - 539s 2s/step - loss: 1.2808 - accuracy: 0.5527
Epoch 4/10
350/350 [==============================] - 538s 2s/step - loss: 1.2007 - accuracy: 0.5844
Epoch 5/10
350/350 [==============================] - 540s 2s/step - loss: 1.1266 - accuracy: 0.6117
Epoch 6/10
350/350 [==============================] - 532s 2s/step - loss: 1.0594 - accuracy: 0.6365
Epoch 7/10
350/350 [==============================] - 529s 2s/step - loss: 1.0095 - accuracy: 0.6507
Epoch 8/10
350/350 [==============================] - 535s 2s/step - loss: 0.9656 - accuracy: 0.6684
Epoch 9/10
350/350 [==============================] - 553s 2s/step - loss: 0.9206 - accuracy: 0.6860
Epoch 10/10
350/350 [============================

In [ ]:
AlexNet.save('CNN_CIFAR.h5')

In [ ]:
from keras.models import Sequential, load_model
model = load_model('CNN_CIFAR.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization_9 (Batch (None, 8, 8, 96)          384       
_________________________________________________________________
activation_9 (Activation)    (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_10 (Activation)   (None, 4, 4, 256)        

# Freezing all the Top layers

In [ ]:
# Freeze all the layers
for layer in model.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f77dd94ff90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f77dad6fd50> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f77dd959310> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f77dad93310> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f77dc78ca10> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f77dd9488d0> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f77dd951450> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f77dc77ad90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f77dace5c90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f77dace54d0> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f77dace0b10> False
<tensorflow.python.keras.layers.con

# Fine Tuning the Dense Layer

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)
# Create the model
model1 = Sequential()

# Add the vgg convolutional base model
model1.add(model)

# Add new layers
model1.add(Flatten(input_shape=model.output_shape[1:]))
model1.add(Dense(120, activation ='relu'))
model1.add(Dense(84, activation ='relu'))
model1.add(Dense(10, activation ='softmax'))
model1.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr = 0.01),
              metrics =['accuracy'])
# Show a summary of the model. Check the number of trainable parameters
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 1)                 20863529  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 120)               240       
_________________________________________________________________
dense_6 (Dense)              (None, 84)                10164     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                850       
Total params: 20,874,783
Trainable params: 13,303
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
model1.compile(loss='categorical_crossentropy', 
     optimizer='adam',
     metrics=['acc'])

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [ ]:
#Training the model
model1.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
350/350 [==============================] - 107s 305ms/step - loss: 1.5940 - acc: 0.4951 - val_loss: 1.1833 - val_acc: 0.5897
Epoch 2/10
350/350 [==============================] - 76s 218ms/step - loss: 1.0226 - acc: 0.6493
Epoch 3/10
350/350 [==============================] - 76s 218ms/step - loss: 0.9663 - acc: 0.6710
Epoch 4/10
350/350 [==============================] - 76s 218ms/step - loss: 0.9391 - acc: 0.6832
Epoch 5/10
350/350 [==============================] - 76s 217ms/step - loss: 0.9117 - acc: 0.6915
Epoch 6/10
350/350 [==============================] - 77s 219ms/step - loss: 0.8961 - acc: 0.7008
Epoch 7/10
350/350 [==============================] - 76s 217ms/step - loss: 0.8872 - acc: 0.7025
Epoch 8/10
350/350 [==============================] - 76s 216ms/step - loss: 0.8872 - acc: 0.7002
Epoch 9/10
350/350 [==============================] - 79s 224ms/step - loss: 0.8714 - acc: 0.7058
Epoch 10/10
350/350 [==============================] - 76s 217ms/step - loss: 0.

# Fine-Tuning for the small dataset and similar to original dataset

In [11]:
import tensorflow_datasets as tfds
import numpy as np
import tensorflow as tf
from tensorflow import keras
tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:10%]", "train[10%:20%]", "train[10%:20%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Number of training samples: 2326
Number of validation samples: 2326
Number of test samples: 2326


In [12]:
size = (224, 224)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [14]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

In [17]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
x_val = x_train[-2000:,:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:,:]
y_train = y_train[:-2000]

In [18]:
from tensorflow.keras import datasets, layers, models, losses
model = models.Sequential()
model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))
model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_1 (Resizing)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34944     
_________________________________________________________________
lambda (Lambda)              (None, 56, 56, 96)        0         
_________________________________________________________________
activation (Activation)      (None, 56, 56, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 256)         614656    
_________________________________________________________________
lambda_1 (Lambda)            (None, 7, 7, 256)        

In [20]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
907/907 [==============================] - 188s 206ms/step - loss: 0.9138 - accuracy: 0.6075 - val_loss: 0.3385 - val_accuracy: 0.8790
Epoch 2/10
907/907 [==============================] - 187s 206ms/step - loss: 0.3449 - accuracy: 0.8733 - val_loss: 0.2902 - val_accuracy: 0.8935
Epoch 3/10
907/907 [==============================] - 186s 206ms/step - loss: 0.2878 - accuracy: 0.8969 - val_loss: 0.2607 - val_accuracy: 0.9025
Epoch 4/10
907/907 [==============================] - 187s 206ms/step - loss: 0.2555 - accuracy: 0.9070 - val_loss: 0.3275 - val_accuracy: 0.8900
Epoch 5/10
907/907 [==============================] - 187s 206ms/step - loss: 0.2485 - accuracy: 0.9104 - val_loss: 0.2662 - val_accuracy: 0.9050
Epoch 6/10
907/907 [==============================] - 188s 208ms/step - loss: 0.2212 - accuracy: 0.9186 - val_loss: 0.2804 - val_accuracy: 0.9095
Epoch 7/10
907/907 [==============================] - 187s 206ms/step - loss: 0.1986 - accuracy: 0.9275 - val_loss: 0.2468 -

In [21]:
# Freeze all the layers
for layer in model.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.layers.preprocessing.image_preprocessing.Resizing object at 0x7fbb8bb38bd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbb8be9a650> False
<tensorflow.python.keras.layers.core.Lambda object at 0x7fbb8bb50e90> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fbb8bb25f10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbb8bb25e90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbb8cbc4e50> False
<tensorflow.python.keras.layers.core.Lambda object at 0x7fbb8cbdc590> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fbb8cbd7690> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbb8bbc56d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbb8cba4e10> False
<tensorflow.python.keras.layers.core.Activation object at 0x7fbb8bb4b750> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbb8bb5065

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)
# Create the model
model1 = Sequential()

# Add the vgg convolutional base model
model1.add(model)

# Add new layers
model1.add(Flatten(input_shape=model.output_shape[1:]))
model1.add(Dense(120, activation ='relu'))
model1.add(Dense(84, activation ='relu'))
model1.add(Dense(10, activation ='softmax'))
model1.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr = 0.01),
              metrics =['accuracy'])
# Show a summary of the model. Check the number of trainable parameters
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 10)                21622154  
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               1320      
_________________________________________________________________
dense_4 (Dense)              (None, 84)                10164     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                850       
Total params: 21,634,488
Trainable params: 12,334
Non-trainable params: 21,622,154
_________________________________________________________________


In [23]:
model1.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model1.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
907/907 [==============================] - 79s 86ms/step - loss: 0.5684 - accuracy: 0.9199 - val_loss: 0.2664 - val_accuracy: 0.9250
Epoch 2/10
907/907 [==============================] - 78s 86ms/step - loss: 0.1536 - accuracy: 0.9500 - val_loss: 0.2642 - val_accuracy: 0.9235
Epoch 3/10
907/907 [==============================] - 79s 87ms/step - loss: 0.1514 - accuracy: 0.9493 - val_loss: 0.2644 - val_accuracy: 0.9230
Epoch 4/10
907/907 [==============================] - 79s 87ms/step - loss: 0.1459 - accuracy: 0.9504 - val_loss: 0.2696 - val_accuracy: 0.9225
Epoch 5/10
907/907 [==============================] - 78s 86ms/step - loss: 0.1453 - accuracy: 0.9503 - val_loss: 0.2522 - val_accuracy: 0.9225
Epoch 6/10
907/907 [==============================] - 78s 86ms/step - loss: 0.1431 - accuracy: 0.9513 - val_loss: 0.2547 - val_accuracy: 0.9250
Epoch 7/10
907/907 [==============================] - 78s 86ms/step - loss: 0.1461 - accuracy: 0.9492 - val_loss: 0.2510 - val_accuracy: